In [1]:
import warnings

warnings.filterwarnings("ignore")

import itertools
import json
import math
import os
import re
import sys
from collections import Counter
from copy import deepcopy
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from rich import print
from tqdm import tqdm

In [2]:
def plot_mtx(mtx, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.matshow(mtx, cmap=plt.cm.Blues)
    plt.plot()

In [3]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """Convert a list of string to a list of other types

    pattern: a list of types for one item.
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [4]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" ", start_row=None, end_row=None
)


print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore 
and 5 clay. Each geode robot costs 3 ore and 18 obsidian. | Last line: Blueprint 30: Each ore robot costs 2 ore. 
Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 10 clay. Each geode robot costs 2 ore and 11 
obsidian.

------

Convert to:

First line: ['Blueprint', '1:', 'Each', 'ore', 'robot', 'costs', '4', 'ore.', 'Each', 'clay', 'robot', 'costs', 
'4', 'ore.', 'Each', 'obsidian', 'robot', 'costs', '3', 'ore', 'and', '5', 'clay.', 'Each', 'geode', 'robot', 
'costs', '3', 'ore', 'and', '18', 'obsidian.']

Last line: ['Blueprint', '30:', 'Each', 'ore', 'robot', 'costs', '2', 'ore.', 'Each', 'clay', 'robot', 'costs', 
'2', 'ore.', 'Each', 'obsidian', 'robot', 'costs', '2', 'ore', 'and', '10', 'clay.', 'Each', 'geode', 'robot', 
'costs', '2', 'ore', 'and', '11', 'obsidian.']

------

In [5]:
class Blueprint:
    def __init__(self, input_li):
        self.num = int(input_li[1].split(":")[0])
        # Cost: [ore, clay, obsidian, geode]
        self.ore_cost = np.array([int(input_li[6]), 0, 0, 0], dtype="int")
        self.clay_cost = np.array([int(input_li[12]), 0, 0, 0], dtype="int")
        self.obsidian_cost = np.array(
            [int(input_li[18]), int(input_li[21]), 0, 0], dtype="int"
        )
        self.geode_cost = np.array(
            [int(input_li[27]), 0, int(input_li[30]), 0], dtype="int"
        )
        # Robot count: [ore, clay, obsidian,geode]
        self.robot = np.array([1, 0, 0, 0], dtype="int")
        # Robot count: [ore, clay, obsidian, geode]
        self.resource = np.array([0, 0, 0, 0], dtype="int")
        self.time = 0
        self.open_geode = 0

    def __str__(self):
        return f"Current time: {self.time}\nOpened geodes: {self.open_geode}\nRobot Number: {self.robot}\nResource: {self.resource}\n"

    def one_more_min(self):
        self.time += 1
        robot_start = self.robot.copy()
        
        # Build robots
        for idx, cost in enumerate(
            [self.geode_cost, self.obsidian_cost, self.clay_cost, self.ore_cost]
        ):
            if np.all(self.resource >= cost):
                # Only build 1 robot each time
                build_flag = (
                    (idx == 0) # Build as many geode as possible
                    or ((np.random.rand() > 0.2) and (idx == 1)) # Build randomly obsidian (80%)
                    or ((np.random.rand() > 0.4) and (idx > 1)) # Build randomly clay/ore (60%)
                )
                if build_flag:
                    self.robot[3 - idx] += 1
                    self.resource -= cost
                    break

        # Produce by old robots
        self.resource += robot_start
        self.open_geode += robot_start[-1]

In [6]:
%%time

try_time = 9999

build_time = 24
res_sum = 0

for info_li in tqdm(data):
    res_li = []
    for _ in range(try_time):
        bp = Blueprint(info_li)
        for _ in range(build_time):
            bp.one_more_min()
        res_li.append(bp.open_geode)

    res_sum += max(res_li) * bp.num

print(f"Answer to Q1: {res_sum}")

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [02:06<00:00,  4.23s/it]


Answer to Q1: 1653

CPU times: total: 2min 6s
Wall time: 2min 6s


In [7]:
%%time

try_time = 99999

build_time = 32
res_sum = 1

for info_li in tqdm(data[:3]):
    res_li = []
    for _ in range(try_time):
        bp = Blueprint(info_li)
        for _ in range(build_time):
            bp.one_more_min()
        res_li.append(bp.open_geode)

    res_sum *= max(res_li)

print(f"Answer to Q2: {res_sum}")

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:05<00:00, 41.84s/it]


Answer to Q2: 4212

CPU times: total: 2min 5s
Wall time: 2min 5s
